In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
vacation = pd.read_csv("output_file")
vacation.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Sao Felix do Xingu,100,BR,1578361714,97,-6.64,-51.99,70.43,0.94
1,1,Victoria,75,BN,1578361287,74,5.28,115.24,84.20,3.36
2,2,Port Elizabeth,1,US,1578361579,51,39.31,-74.98,45.00,7.63
3,3,Vaini,75,IN,1578361482,94,15.34,74.49,68.00,9.17
4,4,Qaanaaq,32,GL,1578361562,84,77.48,-69.36,-18.02,11.54


In [3]:
gmap = gmaps.configure(api_key=g_key)
locations = vacation[['Lat', 'Lng']]
weights = vacation['Humidity'] == 100
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
hotel_df = vacation[((vacation["Max Temp"]>72) & (vacation["Max Temp"]<80) & (vacation["Wind Speed"]<10) & (vacation["Cloudiness"]==0))]
hotel_df = hotel_df.rename(columns={"Unnamed: 0": "Index"})
del hotel_df["Index"]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
90,Port Lincoln,0,AU,1578361667,52,-34.72,135.86,72.97,8.57
177,Morondava,0,MG,1578361587,88,-20.30,44.28,75.51,1.14
256,Salaga,0,GH,1578361732,29,8.55,-0.52,76.86,4.56
282,Hosangadi,0,IN,1578361733,73,13.70,74.96,75.90,3.24
393,Pathein,0,MM,1578361742,57,16.78,94.73,77.99,3.11
518,Ilhabela,0,BR,1578361291,81,-23.78,-45.36,78.91,9.71
519,Leticia,0,CO,1578361750,94,-4.21,-69.94,78.80,3.15


In [5]:
locations = []
params = {
    "types": "Hotel",
    "keyword": "Lodging",
    "radius": 5000,
    "key": g_key,
    }
dropped_rows = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    if 'status' in response and response['status']== "OK":
        locations.append(response['results'][0]['name'])
    else:
        dropped_rows.append(index)
hotel_df = hotel_df.drop(dropped_rows)
hotel_df["Hotel Name"] = locations

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
90,Port Lincoln,0,AU,1578361667,52,-34.72,135.86,72.97,8.57,Hilton Motel
177,Morondava,0,MG,1578361587,88,-20.30,44.28,75.51,1.14,Palissandre Cote Ouest resort & SPA
256,Salaga,0,GH,1578361732,29,8.55,-0.52,76.86,4.56,Maridon Lodge
393,Pathein,0,MM,1578361742,57,16.78,94.73,77.99,3.11,The First Hotel
518,Ilhabela,0,BR,1578361291,81,-23.78,-45.36,78.91,9.71,Pousada Vila das Velas
519,Leticia,0,CO,1578361750,94,-4.21,-69.94,78.80,3.15,Waira Suites Hotel


In [6]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))